In [ ]:
import pymongo
import pandas as pd
import numpy as np
import json


from tqdm.auto import tqdm

In [ ]:
host = 'testnet.toncenter.com'
with open('../private/mongodb_password', 'r') as f:
    password = f.read()

client = pymongo.MongoClient(f'mongodb://user1:{password}@{host}:27017/test?authSource=admin&readPreference=primary&directConnection=true&ssl=false')
client

In [ ]:
host_set = set()

req = {
    'request.headers.host': 'testnet.toncenter.com', 
    'response.status_code': {"$ne": 200}
}
total = client.pyton.requests.count_documents({})
count = client.pyton.requests.count_documents(req)
it = client.pyton.requests.find({})

if total > 0:
    print(f'Errors: {count} / {total} = {count / total * 100:.2f}%')
data = list(tqdm(it, total=count))

In [ ]:
stats_list = []
for x in data:
    body = x['request']['body']
    try:
        body = json.loads(x['request']['body'])
    except:
        body = {}
    stats_list.append({
        'url': x['request']['url'], 
        'method': body.get('method'),
        'status_code': x['response']['status_code'],
        'content_type': x['request']['headers'].get('content-type')
    })

In [ ]:
stats = pd.DataFrame(stats_list)
bad_ct = stats[stats.content_type == 'text/plain;charset=UTF-8']
print(f'Bad content_type: {bad_ct.shape[0]} / {stats.shape[0]} = {bad_ct.shape[0] / stats.shape[0] * 100:.2f}%')

In [ ]:
stats = stats[stats.content_type != 'text/plain;charset=UTF-8']
stats[['method', 'status_code']].value_counts().sort_index()

In [ ]:
for x in data:
    status_code = x['response']['status_code']
    content_type = x['request']['headers'].get('content-type')
    if status_code == 200:
        continue
    if status_code == 422 and content_type == 'text/plain;charset=UTF-8':
        continue
    body = x['request']['body']
    try:
        req_body = json.loads(x['request']['body'])
        res_body = json.loads(x['response']['body'])
    except Exception as ee:
        print(f'Failed to parse body: {ee}')
        continue
    if 'TypeError: __init__() takes exactly 3' in res_body['error']:
        continue
    print('=' * 100)
    print('status_code:', status_code)
    display(x['request'])
    print(res_body.get('error'))